In [3]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations as albu
import torch
import segmentation_models_pytorch as smp
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib
import json

In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

In [5]:
DATA_DIR = './'

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'test')

suffix = 'test_real'

In [6]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [7]:
class HubMapDataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['unlabelled', 'blood_vessel']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return self.images_fps[i], image, mask
        
    def __len__(self):
        return len(self.ids)

In [8]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=512, min_width=352, always_apply=True, border_mode=0),
        albu.RandomCrop(height=512, width=352, always_apply=True),

        albu.GaussNoise(p=0.2),
        albu.Perspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightnessContrast(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.Sharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomBrightnessContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [9]:
CLASSES = ['unlabelled', 'blood_vessel']
ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'

In [10]:
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [11]:
# Inference
from collections import OrderedDict
data_parallel_model = True
model_path = './models/unet-dataset1-batch-size-8-models/best_model_unet_dataset1.pth'
if data_parallel_model:
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    checkpoint_state = checkpoint.state_dict()
    modified_checkpoint_state = OrderedDict()
    for key, value in checkpoint_state.items():
      new_key = key[len('module.'):]
      modified_checkpoint_state[new_key] = value
    best_model = torch.load('./models/starter_model.pth', map_location=torch.device('cpu'))
    best_model.load_state_dict(modified_checkpoint_state)
    best_model = best_model.to(DEVICE)
else:
    best_model = torch.load(model_path, map_location=torch.device('cpu'))
    best_model = best_model.to(DEVICE)

In [12]:
CLASSES = ['unlabelled', 'blood_vessel']
test_dataset = HubMapDataset(
    x_test_dir, 
    y_test_dir, 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_loader = DataLoader(test_dataset, batch_size=4)

In [13]:
import cv2
import numpy as np
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def extract_polygon_masks(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    masks = []
    
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if approx.shape[0] >= 3:
            polygon_mask = np.zeros_like(mask)
            cv2.drawContours(polygon_mask, [approx], 0, 1, -1)
            masks.append(polygon_mask.astype('bool'))
    
    return masks

def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool_:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)
  
  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

In [14]:
import time
def generate_submission(model, device, dataloader, suffix):
    model.eval()
    num_batches = len(dataloader)
    print(f'Processing a total of {num_batches} images for submission')
    submission_dicts = []
    start_time = time.time()
    # Disable gradient calculation
    with torch.no_grad():
        # Iterate over the validation dataset
        for batch_idx, (img_file, inputs, targets) in enumerate(dataloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            softmaxed_outputs = torch.softmax(outputs, dim=1)
            outputs = softmaxed_outputs[:,1,:,:].numpy()
            outputs_thresh = (outputs > 0.5).astype('uint8')
            for i in range(len(outputs_thresh)):
              cur_dict = dict()
              img_id = img_file[i].split('/')[-1].split('.')[0]
              cur_dict['id'] = img_id
              cur_dict['height'] = 512
              cur_dict['width'] = 512
              prediction_string = ''
              polygon_masks = extract_polygon_masks(outputs_thresh[i,:,:])
              for polygon_mask in polygon_masks:
                polygon_mask_conf = round(((polygon_mask * outputs[i,:,:]).sum())/(polygon_mask.sum()), 2)
                polygon_mask_string = encode_binary_mask(polygon_mask.astype('bool')).decode('utf-8')
                prediction_string += f'0 {polygon_mask_conf} {polygon_mask_string} '
              cur_dict['prediction_string'] = prediction_string.strip()
              submission_dicts.append(cur_dict)
            if batch_idx % 10 == 0:
              print(f'On batch {batch_idx} and finished in {float(time.time()-start_time)/60} minutes')
        submission_df = pd.DataFrame.from_dict(submission_dicts)
        submission_df.to_csv(f'./submissions/submission_{suffix}.csv', index=False)

In [15]:
if not os.path.exists('./submissions'):
  os.mkdir('./submissions')

In [16]:
generate_submission(best_model, DEVICE, test_loader, suffix)

Processing a total of 1 images for submission
On batch 0 and finished in 0.028425848484039305 minutes


In [ ]:
def generate_ground_truth_map_files(tiles_dicts_new, x_test_dir, suffix):
  bbox_dicts = []
  segfile_dicts = []
  labels_info = set()
  img_width = 512
  img_height = 512
  print(f'Processing a total of {len(tiles_dicts_new)} tiles')
  start_time = time.time()
  for idx, tiles_dict in enumerate(tiles_dicts_new):
    img_id = tiles_dict['id']
    base_image = cv2.imread(f'{x_test_dir}/{img_id}.png')
    for annot in tiles_dict['annotations']:
      if annot['type'] == 'blood_vessel':
        blood_vessel_masked_image = np.zeros((512, 512))
        cur_dict = dict()
        cur_segfile_dict = dict()
        cur_dict['ImageID'] = img_id
        cur_dict['LabelName'] = annot['type']
        coords = annot['coordinates'][0]
        cv2.fillPoly(blood_vessel_masked_image, pts=[np.array(coords)], color=1)
        encoded_mask = encode_binary_mask(blood_vessel_masked_image.astype('bool')).decode('utf-8')
        x_vals = [x[0] for x in coords]
        y_vals = [x[1] for x in coords]
        x_min = float(min(x_vals))/img_width
        x_max = float(max(x_vals))/img_width
        y_min = float(min(y_vals))/img_height
        y_max = float(max(y_vals))/img_height
        cur_dict['XMin'] = x_min
        cur_dict['XMax'] = x_max
        cur_dict['YMin'] = y_min
        cur_dict['YMax'] = y_max
        cur_dict['IsGroupOf'] = 0
        cur_segfile_dict['ImageID'] = img_id
        cur_segfile_dict['LabelName'] = annot['type']
        cur_segfile_dict['ImageWidth'] = img_width
        cur_segfile_dict['ImageHeight'] = img_height
        cur_segfile_dict['XMin'] = x_min
        cur_segfile_dict['XMax'] = x_max
        cur_segfile_dict['YMin'] = y_min
        cur_segfile_dict['YMax'] = y_max
        cur_segfile_dict['IsGroupOf'] = 0
        cur_segfile_dict['Mask'] = encoded_mask
        bbox_dicts.append(cur_dict)
        segfile_dicts.append(cur_segfile_dict)
        cur_labels_info = (img_id, annot['type'], 1)
        if cur_labels_info not in labels_info:
          labels_info.add(cur_labels_info)
    if idx % 50 == 0:
      print(f'Finished {idx} tiles in {float(time.time()-start_time)/60} minutes')
  bbox_dicts_df = pd.DataFrame.from_dict(bbox_dicts)
  bbox_dicts_df.to_csv(f'./map_input_data/segmentation_bbox_{suffix}.csv', index=False)
  labels_info_df = pd.DataFrame(list(labels_info), columns=['ImageID', 'LabelName', 'Confidence'])
  labels_info_df.to_csv(f'./map_input_data/segmentation_labels_{suffix}.csv', index=False)
  segfile_df = pd.DataFrame.from_dict(segfile_dicts)
  segfile_df.to_csv(f'./map_input_data/segmentation_masks_{suffix}.csv', index=False)

In [ ]:
if not os.path.exists('./map_input_data'):
  os.mkdir('./map_input_data')

In [ ]:
with open(f'./polygons.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
tiles_dicts = []
for json_str in json_list:
    tiles_dicts.append(json.loads(json_str))

In [ ]:
x_test_dir

In [ ]:
tgt_ids = [x.split('.')[0] for x in os.listdir(x_test_dir)]
tiles_dicts_new = [x for x in tiles_dicts if x['id'] in tgt_ids]

In [ ]:
generate_ground_truth_map_files(tiles_dicts_new, x_test_dir, suffix)

In [ ]:
def create_pred_parts(pred_string):
  pred_parts = pred_string.split()
  pred_parts_arr = []
  for i in range(0, len(pred_parts), 3):
    pred_parts_arr.append(pred_parts[i:i+3])
  return pred_parts_arr

In [ ]:
def generate_prediction_map_file(submission_df, suffix):
  non_empty_pred_mask = submission_df.apply(lambda x: x['prediction_string']!='', axis=1)
  submission_df = submission_df[non_empty_pred_mask].dropna(subset=['prediction_string'], axis=0)
  submission_df = submission_df.rename(columns={'id': 'ImageID', 'height': 'ImageHeight', 'width': 'ImageWidth'})
  submission_df['prediction_string'] = submission_df['prediction_string'].apply(create_pred_parts)
  submission_df = submission_df.explode('prediction_string')
  submission_df[['LabelName', 'Score', 'Mask']] = submission_df['prediction_string'].apply(pd.Series)
  submission_df['LabelName'] = submission_df['LabelName'].astype(str).replace('0', 'blood_vessel')
  submission_df = submission_df.drop('prediction_string', axis=1)
  submission_df.to_csv(f'./map_input_data/seg_preds_{suffix}.csv', index=False)

In [ ]:
submission_df = pd.read_csv(f'./submissions/submission_{suffix}.csv')
generate_prediction_map_file(submission_df, suffix)